In [ ]:
# /data/QAAPI/qavenv/bin/activate

In [2]:
import os
import pandas as pd
# from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import CountVectorizer

import pdfplumber
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
import requests

# from Classification_Api import *
from pdf_boq_extraction2 import *
from doc_boq_extraction import get_most_relevant_table_doc


def should_drop(value):
    if isinstance(value, str):
        return value.startswith("number #")
    return False


def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i) < 128)

def normalize_text(text):
    if isinstance(text, str):
        return ' '.join(text.lower().split())
    return text  # Return numeric values or other types as-is

def save_all_dfs(dfs, base_path, base_name):
    os.makedirs(base_path, exist_ok=True)
    
    file_number = 0
    for df in dfs:
        # # Clean the DataFrame
        # df = set_header_from_keywords(df, keywords)
        # df = df.dropna(how='all')
        # df = df.dropna(axis=1, how='all')
        while True:
            output_file = os.path.join(base_path, f"{base_name}_BOQ{file_number if file_number > 0 else ''}.csv")
            if not os.path.exists(output_file):
                df.to_csv(output_file, index=False)
                print(f"Saved extracted data to: {output_file}")
                break
            file_number += 1

def find_header_row_1608(df, header_keywords, min_matches=2):
    for i, row in df.iterrows():
        row_str = ' '.join(row.dropna().astype(str).tolist()).lower()

        # Check individual keyword matches
        keyword_matches = [any(fuzz.partial_ratio(keyword, word) > 40 for word in row_str.split()) for keyword in header_keywords]
        if sum(keyword_matches) >= min_matches:  # Heuristic: At least 2-3 keywords should match
            return i
    return None

def name_check_1609_doc(folder_path, base_path):
    file_names = os.listdir(folder_path)
    valid_extensions = ('.xlsx', '.csv', '.pdf', '.xls','.doc','.docx')
    matched_files = [file for file in file_names if file.lower().endswith(valid_extensions)]
    print("Matched files:", matched_files)
    
    all_dfs = []  # List to store DataFrames
    for file_name in matched_files:
        try:
            file_path = os.path.join(folder_path, file_name)
            df = None

            if file_name.lower().endswith('.xlsx') or file_name.lower().endswith('.xls'):
                if file_name.lower().endswith('.xls'):
                    df = pd.read_excel(file_path, engine='xlrd')
                else:
                    df = pd.read_excel(file_path)
                print("XLSX:", file_name)
                
            elif file_name.lower().endswith('.csv'):
                df = pd.read_csv(file_path, encoding='ISO-8859-1')
                print("CSV:", file_name)
                
            elif file_name.lower().endswith('.pdf'):
                df = get_most_relevant_table(file_path)
                if df is not None:
                    print(f"Processed PDF: {file_name}")
                    if len(all_dfs) > 1:  # If all_dfs is empty
                        if 'RA Details' in df.columns:
                            print(f"Skipping file with 'RA Details' column: {file_name}")
                            continue  # Skip appending this DataFrame
                    else:                         
                        all_dfs.append(df)  # Append the extracted DataFrame to all_dfs
                else:
                    print(f"Failed to extract table from PDF: {file_name}")
                    continue
                
            else:
                print("Other file type:", file_name)
                continue

            if df is not None and not file_name.lower().endswith('.pdf'):
                header_keywords = [
                    'bill of quantity', 'qty', 'quantity', "inr", "item/activity", 'total price', "item description", "weight",
                    'quantity', 'qty', 'quintal', 'quantity in quintal', 'total quantity', 'rate', 'price', 'base rate',
                    'offer rate', 'rate per quintal', 'rate in rs.', 'rate per unit', 'item rate', 'rate per item',
                    'price per unit', 'price', 'schedule of rates', 'charges', 'total cost', 'cost', 'total base cost',
                    'total offer cost', 'price in rs.', 'overall rate', 'aggregate rate', 'extended amount', 'landed cost',
                    'final rate', 'total charges', 'lumpsum cost', 'lump-sum', 'item', 'description', 'particulars',
                    'activity', 'item/activity', 'deliverables', 'item description', 'item name', 'item title', 'item description'
                ]
                
                # header_row_idx = find_header_row_1608(df, header_keywords)
                
                if header_row_idx is not None:
                    if header_row_idx != 0:
                        # Only modify the DataFrame if the header is not in the first row
                        df.columns = df.iloc[header_row_idx]
                        df = df[header_row_idx + 1:].reset_index(drop=True)
                    
                    df = df.apply(lambda x: x.map(normalize_text) if x.dtype == 'object' else x)
                    
                    # Ensure specific columns are within the range of DataFrame columns
                    specific_cols = [0, 1, 3, 4, 5, 12, 52, 53, 54]
                    max_index = df.shape[1] - 1
                    valid_cols = [col for col in specific_cols if col <= max_index]

                    if valid_cols:
                        df = df.iloc[:, valid_cols]
                        rows_to_drop = df.apply(lambda x: x.map(should_drop).any(), axis=1)


                        # Get the index of the first row that meets the condition
                        first_row_to_drop = rows_to_drop.idxmax() if rows_to_drop.any() else None

                        # Drop the row and all rows above it if a matching row is found
                        if first_row_to_drop is not None:
                            df = df.drop(index=df.index[:first_row_to_drop + 1])
                            print(f"Dropped rows up to {first_row_to_drop} in file: {file_name}")
                        else:
                            df = df.copy()
                
                        # Replace 'Unnamed' columns with '-'
                        df.columns = [str(col) for col in df.columns] 
                        df.columns = ['-' if 'Unnamed' in col else col for col in df.columns]
                        df.dropna(axis=1, how='all', inplace=True)  # Drop empty columns
                        df.dropna(axis=0, how='all', inplace=True)  # Drop empty rows
                        all_dfs.append(df)
                    
                    else:
                        print(f"No valid columns found in file: {file_name}")
                
                else:
                    print(f"Header row not found in file: {file_name}")

        except pd.errors.ParserError:
            print(f"Error reading file: {file_name}")
        except Exception as e:
            print(f"Error reading file: {file_name} - {e}")
    # Check if all_dfs is empty and process DOC/DOCX files if present
    if not all_dfs:
        doc_files = [file for file in file_names if file.lower().endswith(('.doc', '.docx'))]
        for doc_file in doc_files:
            file_path = os.path.join(folder_path, doc_file)
            df = get_most_relevant_table_doc(file_path)
            if df is not None:
                all_dfs.append(df)
                print(f"Extracted table from DOC/DOCX: {doc_file}")
            else:
                print(f"Failed to extract table from DOC/DOCX: {doc_file}")
    save_all_dfs(all_dfs, base_path, os.path.basename(folder_path))


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
/data/unzipdocument/dailydocument_23-10-24/73342080